In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import altair as alt
import pickle
import string
import spacy
import nltk 
import re

from sklearn.naive_bayes import *
from sklearn.ensemble import *
from sklearn.neighbors import *
from sklearn.tree import *
from sklearn.calibration import *
from sklearn.linear_model import *
from sklearn.multiclass import *
from sklearn.svm import *
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from nltk.stem import WordNetLemmatizer 
from collections import Counter

from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, auc, roc_curve
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline

In [ ]:
import spacy
try:
    nlp = spacy.load('en_core_web_lg')
except Exception as e:
    !spacy download en_core_web_lg
    nlp = spacy.load('en_core_web_lg')

In [ ]:
df = pd.read_csv('/content/overview-of-recordings.csv')

In [ ]:
df.head()

In [ ]:
df_text = df[['phrase', 'prompt']]

In [ ]:
df_text.isna().sum()
df_text.dropna(inplace = True)

In [ ]:
df_text

In [ ]:
df.duplicated().sum()

In [ ]:
negative = ["haven't", "won't", "mightn't", "not", "doesn't", "needn't", "wouldn'", "weren't", "didn't", "hadn't", "wouldn't", "wasn't", "don't", "isn'", "no", "shan't", "aren't", "couldn't", "mustn't", "cannot"]
STOP_WORDS = set([i for i in STOP_WORDS if i not in negative])
df_text['no_of_characters'] = df_text['phrase'].apply(len)
df_text['no_of_tokens'] = df_text['phrase'].apply(lambda w: len(w.split()))
df_text['average_length_of_word'] = df_text['phrase'].apply(lambda w: np.mean([len(a) for a in w.split()]))
df_text['no_stop_words'] = df_text['phrase'].apply(lambda w: len([t for t in w.split() if t not in STOP_WORDS]))
df_text

In [ ]:
def clean_txt(docs):
    speech_words = nlp(docs)
    lower_text = [w.text.lower() for w in speech_words]
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('', w) for w in lower_text]
    words = [word for word in stripped if word.isalpha()]
    words = [w for w in words if not w in  list(STOP_WORDS)]
    words = [word for word in words if len(word) > 2]
    lem_words = [word.lemma_ for word in nlp(' '.join(words))]
    combined_text = ' '.join(lem_words)
    return combined_text

# Cleaning the text data
df_text['cleaned_phrase'] = df_text['phrase'].apply(clean_txt)
df_text

In [ ]:
df_text

In [ ]:
# Spot-Check Normalized Text Models
def NormalizedTextModel(nameOfvect):
    if nameOfvect == 'countvect':
        vectorizer = CountVectorizer()
    elif nameOfvect =='tfvect':
        vectorizer = TfidfVectorizer()
    elif nameOfvect == 'hashvect':
        vectorizer = HashingVectorizer()

    pipelines = []
    pipelines.append((nameOfvect+'MultinomialNB'  , Pipeline([('Vectorizer', vectorizer),('NB'  , MultinomialNB())])))
    pipelines.append((nameOfvect+'CCCV' , Pipeline([('Vectorizer', vectorizer),('CCCV' , CalibratedClassifierCV())])))
    pipelines.append((nameOfvect+'KNN' , Pipeline([('Vectorizer', vectorizer),('KNN' , KNeighborsClassifier())])))
    pipelines.append((nameOfvect+'CART', Pipeline([('Vectorizer', vectorizer),('CART', DecisionTreeClassifier())])))
    pipelines.append((nameOfvect+'PAC'  , Pipeline([('Vectorizer', vectorizer),('PAC'  , PassiveAggressiveClassifier())])))
    pipelines.append((nameOfvect+'SVM' , Pipeline([('Vectorizer', vectorizer),('RC' , RidgeClassifier())])))
    pipelines.append((nameOfvect+'AB'  , Pipeline([('Vectorizer', vectorizer),('AB'  , AdaBoostClassifier())])  ))
    pipelines.append((nameOfvect+'GBM' , Pipeline([('Vectorizer', vectorizer),('GMB' , GradientBoostingClassifier())])))
    pipelines.append((nameOfvect+'RF'  , Pipeline([('Vectorizer', vectorizer),('RF'  , RandomForestClassifier())])))
    pipelines.append((nameOfvect+'ET'  , Pipeline([('Vectorizer', vectorizer),('ET'  , ExtraTreesClassifier())])))
    pipelines.append((nameOfvect+'SGD'  , Pipeline([('Vectorizer', vectorizer),('SGD'  , SGDClassifier())])))
    pipelines.append((nameOfvect+'OVRC'  , Pipeline([('Vectorizer', vectorizer),('OVRC'  , LogisticRegression())])))
    pipelines.append((nameOfvect+'Bagging'  , Pipeline([('Vectorizer', vectorizer),('Bagging'  , BaggingClassifier())])))
    pipelines.append((nameOfvect+'NN'  , Pipeline([('Vectorizer', vectorizer),('NN'  , MLPClassifier())])))
    return pipelines

# Traing model 
def fit_model(X_train, y_train,models):
    # Test options and evaluation metric
    num_folds = 10
    scoring = 'accuracy'

    results = []
    names = []
    for name, model in models:
        kfold = KFold(n_splits=num_folds)
        cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
        print(msg)

# Split data to training and validation set
def read_in_and_split_data(data, features,target):
    X = data[features]
    y = data[target]
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)
    return X_train, X_test, y_train, y_test

X = 'cleaned_phrase'
target_class = 'prompt'
X_train, X_test, y_train, y_test = read_in_and_split_data(df_text, X, target_class)

In [ ]:
models = NormalizedTextModel('tfvect')
models1 = NormalizedTextModel('hashvect')
fit_model(X_train, y_train, models)
fit_model(X_train, y_train, models1)

In [ ]:
import sklearn
text_clf = Pipeline([('tfidf', TfidfVectorizer()),('bagging', BaggingClassifier(n_estimators=10))])
model = text_clf.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(sklearn.metrics.classification_report(y_test, y_pred))